<a href="https://colab.research.google.com/github/RaminParker/langchain_chrash_course/blob/main/Kategorisierung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain openai

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import pandas as pd
import json

In [ ]:
# Mount your Google Drive on your Google Colab runtime
from google.colab import drive
drive.mount('/content/drive/')

path_to_key = "/content/drive/MyDrive/Colab Notebooks/configs/openai/token.txt" 
file1 = open(path_to_key, "r")
token_oai = file1.read()

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = token_oai

In [ ]:
openai_api_key = token_oai

In [ ]:
# Temp = 0 so that we get clean information without a lot of creativity
chat_model = ChatOpenAI(temperature=0, openai_api_key=openai_api_key, max_tokens=1000)

In [ ]:
# How you would like your reponse structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="input_industry", description="This is the input_industry from the user"),
    ResponseSchema(name="standarized_industry", description="This is the industry you feel is most closely matched to the users input"),
    ResponseSchema(name="match_score",  description="A score 0-100 of how close you think the match is between user input and your match")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (output_parser.get_format_instructions())

In [ ]:
template = """
You will be given a series of industry names from a user.
Find the best corresponding match on the list of standardized names.
The closest match will be the one with the closest semantic meaning. Not just string similarity.

{format_instructions}

Wrap your final output with closed and open brackets (a list of json objects)

input_industry INPUT:
{user_industries}

STANDARDIZED INDUSTRIES:
{standardized_industries}

YOUR RESPONSE:
"""

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)  
    ],
    input_variables=["user_industries", "standardized_industries"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
standardized_industries = ["Transportation & Logistic", "Agriculture", "Consumer Goods", "Hardware & Networking", "Media & Communication", "Corporate Services",
                           "Travel", "Legal", "Wellness & Fitness", "Entertainment", "Public Administration", "Education", "Retail", "Finance", "Health Care"]

In [ ]:
# Your user input

user_input = "air LineZ, airline, aviation, planes that fly, farming, bread, wifi networks, twitter media agency"

_input = prompt.format_prompt(user_industries=user_input, standardized_industries=standardized_industries)


print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

In [ ]:
output = chat_model(_input.to_messages())

In [ ]:
print(type(output))
print(output.content)

In [ ]:
if "```json" in output.content:
    print("hello")
    json_string = output.content.split("```json")[1].strip()
else:
    json_string = output.content

In [ ]:
json_string

In [ ]:
output.content

In [ ]:
# output_parser.parse(output.content) Ideally this works but not in all cases
structured_data = json.loads(json_string)
structured_data